# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
import keras
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [2]:
from tensorflow.keras.layers import Dense, Dropout
from keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential

# Importing cleaned sets

In [3]:
path=r"C:\Users\Sam\Desktop\CF\Machine Learning\Part 2\2.2 Complex ML models and Keras part 1\Dataset-Answers-Weather_Prediction_Pleasant_Weather-CLEANED.csv"

In [4]:
answers=pd.read_csv(os.path.join(path))

In [5]:
pathw=r"C:\Users\Sam\Desktop\CF\Machine Learning\Part 2\2.2 Complex ML models and Keras part 1\Dataset-weather-prediction-dataset-processed-CLEANED.csv"

In [6]:
weather=pd.read_csv(os.path.join(pathw))

In [7]:
answers.shape

(22950, 15)

In [8]:
weather.shape

(22950, 135)

In [9]:
weather.columns

Index(['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure',
       'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine',
       'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max',
       'BELGRADE_cloud_cover',
       ...
       'STOCKHOLM_temp_max', 'VALENTIA_cloud_cover', 'VALENTIA_humidity',
       'VALENTIA_pressure', 'VALENTIA_global_radiation',
       'VALENTIA_precipitation', 'VALENTIA_sunshine', 'VALENTIA_temp_mean',
       'VALENTIA_temp_min', 'VALENTIA_temp_max'],
      dtype='object', length=135)

In [10]:
attributes=[col.replace("BASEL_","") for col in weather.columns[:9]]
attributes

['cloud_cover',
 'humidity',
 'pressure',
 'global_radiation',
 'precipitation',
 'sunshine',
 'temp_mean',
 'temp_min',
 'temp_max']

In [11]:
X=weather.values
X=X.reshape(-1,1,9)

In [12]:
X

array([[[ 7.    ,  0.85  ,  1.018 , ...,  6.5   ,  0.8   , 10.9   ]],

       [[ 1.    ,  0.81  ,  1.0195, ...,  3.7   , -0.9   ,  7.9   ]],

       [[ 4.    ,  0.67  ,  1.017 , ...,  2.4   , -0.4   ,  5.1   ]],

       ...,

       [[ 4.    ,  0.84  ,  1.0263, ...,  1.7   ,  0.7   ,  2.7   ]],

       [[ 5.    ,  0.97  ,  1.0164, ...,  9.7   ,  5.    , 12.6   ]],

       [[ 5.    ,  0.82  ,  1.0142, ..., 10.7   ,  7.9   , 13.5   ]]])

In [14]:
y=answers.values

In [15]:
y=y.reshape(-1,)

In [16]:
y

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
len(y)

344250

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [22]:
def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))
        
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        #model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [23]:
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from keras.models import Sequential
from sklearn.model_selection import cross_val_score
from keras.layers import Conv1D, Conv2D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D

from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)

In [25]:
from keras.wrappers.scikit_learn import KerasClassifier

In [26]:
start = time.time()
params ={
    'neurons': (10, 100),
    'kernel': (1, 3),
    'activation':(0, 9), #9
    'optimizer':(0,7), #7
    'learning_rate':(0.01, 1),
    'batch_size': (200, 1000), #(10, 50), #
    'epochs':(20, 100),
    'layers1':(1,3),
    'layers2':(1,3),
    'normalization':(0,1),
    'dropout':(0,1),
    'dropout_rate':(0,0.3)
}
# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/32
230/230 - 2s - loss: 0.5176 - accuracy: 0.7814 - 2s/epoch - 10ms/step
Epoch 2/32
230/230 - 1s - loss: 0.4679 - accuracy: 0.7801 - 1s/epoch - 5ms/step
Epoch 3/32
230/230 - 1s - loss: 0.2782 - accuracy: 0.8677 - 1s/epoch - 5ms/step
Epoch 4/32
230/230 - 1s - loss: 0.2444 - accuracy: 0.8864 - 1s/epoch - 5ms/step
Epoch 5/32
230/230 - 1s - loss: 0.2287 - accuracy: 0.8932 - 1s/epoch - 5ms/step
Epoch 6/32
230/230 - 1s - loss: 0.2212 - accuracy: 0.8970 - 1s/epoch - 5ms/step
Epoch 7/32
230/230 - 1s - loss: 0.2115 - accuracy: 0.8994 - 1s/epoch - 5ms/step
Epoch 8/32
230/230 - 1s - loss: 0.2047 - accuracy: 0.9035 - 1s/epoch - 5ms/step
Epoch 9/32
230/230 

1722/1722 [==============================] - 2s 1ms/step
Epoch 1/32
230/230 - 2s - loss: 0.4737 - accuracy: 0.7784 - 2s/epoch - 9ms/step
Epoch 2/32
230/230 - 1s - loss: 0.2908 - accuracy: 0.8567 - 1s/epoch - 5ms/step
Epoch 3/32
230/230 - 1s - loss: 0.2475 - accuracy: 0.8833 - 1s/epoch - 5ms/step
Epoch 4/32
230/230 - 1s - loss: 0.2308 - accuracy: 0.8906 - 1s/epoch - 5ms/step
Epoch 5/32
230/230 - 1s - loss: 0.2247 - accuracy: 0.8924 - 1s/epoch - 5ms/step
Epoch 6/32
230/230 - 1s - loss: 0.2162 - accuracy: 0.8971 - 1s/epoch - 5ms/step
Epoch 7/32
230/230 - 1s - loss: 0.2088 - accuracy: 0.9008 - 1s/epoch - 5ms/step
Epoch 8/32
230/230 - 1s - loss: 0.2151 - accuracy: 0.8993 - 1s/epoch - 5ms/step
Epoch 9/32
230/230 - 1s - loss: 0.1958 - accuracy: 0.9072 - 1s/epoch - 5ms/step
Epoch 10/32
230/230 - 1s - loss: 0.1882 - accuracy: 0.9113 - 1s/epoch - 6ms/step
Epoch 11/32
230/230 - 1s - loss: 0.1803 - accuracy: 0.9150 - 1s/epoch - 6ms/step
Epoch 12/32
230/230 - 1s - loss: 0.1736 - accuracy: 0.9182 - 

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Sam\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Sam\anaconda3\Lib\site-packages\keras\wrappers\scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sam\anaconda3\Lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Sam\AppData\Local\Temp\ipykernel_9736\26379690

In [27]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
optimum['activation'] = activationL[round(optimum['activation'])]
optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])
optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','Adam']
optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
             'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
             'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
             'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'activation': 'softsign',
 'batch_size': 961,
 'dropout': 0.7319939418114051,
 'dropout_rate': 0.17959754525911098,
 'epochs': 32,
 'kernel': 1.3119890406724053,
 'layers1': 1,
 'layers2': 3,
 'learning_rate': 0.6051038616257767,
 'neurons': 74,
 'normalization': 0.020584494295802447,
 'optimizer': <keras.optimizers.legacy.ftrl.Ftrl at 0x223af4e76d0>}

In [29]:
epochs = 32
batch_size = 961

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15
layers1 = 1
layers2 = 3
activation = 'softsign'
kernel = 1
neurons = 74
normalization = 0.020584494295802447
dropout = 0.7319939418114051
dropout_rate = 0.17959754525911098
optimizer = 'Adadelta'

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
if normalization > 0.5:
    model.add(BatchNormalization())
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax')) #softmax sigmoid
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #binary_crossentropy

In [31]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/32
287/287 - 4s - loss: 2.4796 - accuracy: 0.4593 - 4s/epoch - 13ms/step
Epoch 2/32
287/287 - 2s - loss: 2.3818 - accuracy: 0.5778 - 2s/epoch - 7ms/step
Epoch 3/32
287/287 - 2s - loss: 2.2702 - accuracy: 0.6499 - 2s/epoch - 7ms/step
Epoch 4/32
287/287 - 2s - loss: 2.1463 - accuracy: 0.6847 - 2s/epoch - 8ms/step
Epoch 5/32
287/287 - 2s - loss: 2.0104 - accuracy: 0.7028 - 2s/epoch - 8ms/step
Epoch 6/32
287/287 - 2s - loss: 1.8648 - accuracy: 0.7160 - 2s/epoch - 7ms/step
Epoch 7/32
287/287 - 2s - loss: 1.7131 - accuracy: 0.7275 - 2s/epoch - 7ms/step
Epoch 8/32
287/287 - 2s - loss: 1.5598 - accuracy: 0.7370 - 2s/epoch - 7ms/step
Epoch 9/32
287/287 - 2s - loss: 1.4104 - accuracy: 0.7466 - 2s/epoch - 7ms/step
Epoch 10/32
287/287 - 2s - loss: 1.2693 - accuracy: 0.7561 - 2s/epoch - 7ms/step
Epoch 11/32
287/287 - 2s - loss: 1.1409 - accuracy: 0.7647 - 2s/epoch - 7ms/step
Epoch 12/32
287/287 - 2s - loss: 1.0275 - accuracy: 0.7723 - 2s/epoch - 7ms/step
Epoch 13/32
287/287 - 2s - loss: 0.9

In [32]:
# trying 100 epochs and 9 classes to see the difference

epochs = 100
batch_size = 961

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 9
layers1 = 1
layers2 = 3
activation = 'softsign'
kernel = 1
neurons = 74
normalization = 0.020584494295802447
dropout = 0.7319939418114051
dropout_rate = 0.17959754525911098
optimizer = 'Adadelta'

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
if normalization > 0.5:
    model.add(BatchNormalization())
for i in range(layers1):
    model.add(Dense(neurons, activation=activation))
if dropout > 0.5:
    model.add(Dropout(dropout_rate, seed=123))
for i in range(layers2):
    model.add(Dense(neurons, activation=activation))
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [33]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/100
287/287 - 3s - loss: 2.2311 - accuracy: 0.1693 - 3s/epoch - 12ms/step
Epoch 2/100
287/287 - 2s - loss: 2.1315 - accuracy: 0.2578 - 2s/epoch - 7ms/step
Epoch 3/100
287/287 - 2s - loss: 2.0153 - accuracy: 0.3706 - 2s/epoch - 7ms/step
Epoch 4/100
287/287 - 2s - loss: 1.8822 - accuracy: 0.5741 - 2s/epoch - 7ms/step
Epoch 5/100
287/287 - 2s - loss: 1.7338 - accuracy: 0.7527 - 2s/epoch - 7ms/step
Epoch 6/100
287/287 - 2s - loss: 1.5707 - accuracy: 0.7970 - 2s/epoch - 8ms/step
Epoch 7/100
287/287 - 2s - loss: 1.4000 - accuracy: 0.7838 - 2s/epoch - 7ms/step
Epoch 8/100
287/287 - 2s - loss: 1.2313 - accuracy: 0.7824 - 2s/epoch - 6ms/step
Epoch 9/100
287/287 - 2s - loss: 1.0776 - accuracy: 0.7838 - 2s/epoch - 7ms/step
Epoch 10/100
287/287 - 2s - loss: 0.9468 - accuracy: 0.7846 - 2s/epoch - 7ms/step
Epoch 11/100
287/287 - 2s - loss: 0.8410 - accuracy: 0.7850 - 2s/epoch - 7ms/step
Epoch 12/100
287/287 - 2s - loss: 0.7579 - accuracy: 0.7852 - 2s/epoch - 7ms/step
Epoch 13/100
287/287 - 2

## 90% accuracy